# Constitutive Model: Hyperelastic Material


Dr. Niraj Kumar Jha
nirajkumar.jha@outlook.com

In [ ]:
import numpy as np
from pylab import *
import scipy.optimize
import pylab 
import matplotlib.pyplot as plt


In [ ]:
"""Neo-Hookean. 3D loading specified by stretches. param[0]=mu, param[1]=kappa"""
def NH_3D(stretch, param):
    L1 = stretch[0]
    L2 = stretch[1]
    L3 = stretch[2]
    F = array([[L1,0,0], [0,L2,0], [0,0,L3]])
    J = det(F)
    Fstar = J**(-1/3) * F
    bstar = dot(Fstar, Fstar.T)
    dev_bstar = bstar - trace(bstar)/3 * eye(3)
    Stress = param[0]/J * dev_bstar + param[1]*(J-1) * eye(3)
    return Stress 

In [ ]:
"""Mooney-Rivlin. 3D loading specified by stretches. param: [C10, C01, kappa]"""
def MR_3D(stretch, param):
    L1 = stretch[0]
    L2 = stretch[1]
    L3 = stretch[2]
    F = array([[L1,0,0], [0,L2,0], [0,0,L3]])
    J = det(F)
    bstar = J**(-2.0/3.0) * dot(F, F.T)
    bstar2 = dot(bstar, bstar)
    I1s = trace(bstar)
    I2s = 0.5 * (I1s**2 - trace(bstar2))
    C10 = param[0]
    C01 = param[1]
    kappa = param[2]
    Stress = 2/J*(C10+C01*I1s)*bstar - 2*C01/J*bstar2 + \
        (kappa*(J-1) - 2*I1s*C10/(3*J) - 4*I2s*C01/(3*J))*eye(3)
    return Stress

In [ ]:
"""Yeoh. 3D loading specified by stretches. param: [C10, C20, C30, kappa]. Returns true stress."""
def Yeoh_3D(stretch, param):
    L1 = stretch[0]
    L2 = stretch[1]
    L3 = stretch[2]
    F = array([[L1,0,0], [0,L2,0], [0,0,L3]])
    J = det(F)
    bstar = J**(-2.0/3.0) * dot(F, F.T)
    devbstar = bstar - trace(bstar)/3 * eye(3)
    I1s = trace(bstar)
    Stress = 2/J*(param[0] + 2*param[1]*(I1s-3) + 3*param[2]*(I1s-3)**2)*devbstar \
        + param[3]*(J-1) * eye(3)
    return Stress

In [ ]:
"""Ogden model. 3D loading specified by stretches. param: [mu1, mu2, ..., alpha1, alpha2, kappa]. Returns true stress."""
def Ogden_3D(stretch, param):
    L1 = stretch[0]
    L2 = stretch[1]
    L3 = stretch[2]  
    F = array([[L1,0,0], [0,L2,0], [0,0,L3]])
    J = L1*L2*L3
    lam = stretch/(J**(1/3))
    N = round((len(param)-1)/2)
    mu = param[0:N]
    alpha = param[N:2*N]
    kappa = param[-1]
    Stress = kappa*(J-1)*eye(3)
    for i in range(N):
        fac = (2/J) * mu[i] / alpha[i]
        tmp = (lam[0]**alpha[i] + lam[1]**alpha[i] + lam[2]**alpha[i]) / 3
        Stress[0,0] = Stress[0,0] + fac * (lam[0]**alpha[i] - tmp)
        Stress[1,1] = Stress[1,1] + fac * (lam[1]**alpha[i] - tmp)
        Stress[2,2] = Stress[2,2] + fac * (lam[2]**alpha[i] - tmp)
    return Stress

In [ ]:
 # Uniaxial stretch = [lam, lam^0.5, lam^0.5]  

lam = np.arange (1, 8.1, 0.1)
Stress_NH = zeros(len(lam))
Stress_MR = zeros(len(lam))
Stress_YH = zeros(len(lam))
Stress_OG = zeros(len(lam))

for i in range(len(lam)):
    stretch = [lam[i], 1/np.sqrt(lam[i]), 1/np.sqrt(lam[i])]
    param_NH = [0.4, 100]
    param_MR = [0.39, 0.015, 100]
    param_YH = [0.4, 0.00004, 0.00004, 100]
    param_OG = [0.62, 0.00118, -0.00981, 1.3, 5, 2, 100]    
    Stress_NH[i] = NH_3D(stretch, param_NH)[0,0] #tension component
    Stress_MR[i] = MR_3D(stretch, param_MR)[0,0] #tension component
    Stress_YH[i] = Yeoh_3D(stretch, param_YH)[0,0] #tension component
    Stress_OG[i] = Ogden_3D(stretch, param_OG)[0,0] #tension component    
    
plt.plot(lam, Stress_NH,color='green', linestyle='dashed', linewidth=2, markersize=12, label='neo-Hookean')
plt.plot(lam, Stress_MR,color='red', linestyle='dashed', linewidth=2, markersize=12, label='Mooney-Rivlin')
plt.plot(lam, Stress_YH,color='magenta', linestyle='dotted', linewidth=2, markersize=12, label='Yeoh')
plt.plot(lam, Stress_OG,color='black', linestyle='solid', linewidth=2, markersize=12, label='Ogden')
plt.xlabel('Stretch')
plt.ylabel('Stress')
plt.title('Uniaxial Experiment')
plt.legend()
plt.show()

 # Biaxial stretch = [lam, lam, 1/(lam)^2]  
for i in range(len(lam)):
    stretch = [lam[i], lam[i], 1/(lam[i]*lam[i])]
    param_NH = [0.4, 100]
    param_MR = [0.39, 0.015, 100]
    param_YH = [0.4, 0.00004, 0.00004, 100]
    param_OG = [0.62, 0.0118, -0.00981, 1.3, 5, -2, 100]    
    Stress_NH[i] = NH_3D(stretch, param_NH)[0,0] #tension component
    Stress_MR[i] = MR_3D(stretch, param_MR)[0,0] #tension component
    Stress_YH[i] = Yeoh_3D(stretch, param_YH)[0,0] #tension component
    Stress_OG[i] = Ogden_3D(stretch, param_OG)[0,0] #tension component    


plt.plot(lam, Stress_NH,color='green', linestyle='dashed', linewidth=2, markersize=12, label='neo-Hookean')
plt.plot(lam, Stress_MR,color='red', linestyle='dashed', linewidth=2, markersize=12, label='Mooney-Rivlin')
plt.plot(lam, Stress_YH,color='magenta', linestyle='dotted', linewidth=2, markersize=12, label='Yeoh')
plt.plot(lam, Stress_OG,color='black', linestyle='solid', linewidth=2, markersize=12, label='Ogden')
plt.xlabel('Stretch')
plt.ylabel('Stress')
plt.title('Biaxial Experiment')
plt.legend()
plt.show()

 # Pure shear stretch = [lam, 1, 1/(lam)]  
for i in range(len(lam)):
    stretch = [lam[i], 1, 1/(lam[i])]
    param_NH = [0.4, 100]
    param_MR = [0.39, 0.015, 100]
    param_YH = [0.4, 0.00004, 0.00004, 100]
    param_OG = [0.62, 0.0118, -0.00981, 1.3, 5, -2, 100]    
    Stress_NH[i] = NH_3D(stretch, param_NH)[0,0] #tension component
    Stress_MR[i] = MR_3D(stretch, param_MR)[0,0] #tension component
    Stress_YH[i] = Yeoh_3D(stretch, param_YH)[0,0] #tension component
    Stress_OG[i] = Ogden_3D(stretch, param_OG)[0,0] #tension component    
    
plt.plot(lam, Stress_NH,color='green', linestyle='dashed', linewidth=2, markersize=12, label='neo-Hookean')
plt.plot(lam, Stress_MR,color='red', linestyle='dashed', linewidth=2, markersize=12, label='Mooney-Rivlin')
plt.plot(lam, Stress_YH,color='magenta', linestyle='dotted', linewidth=2, markersize=12, label='Yeoh')
plt.plot(lam, Stress_OG,color='black', linestyle='solid', linewidth=2, markersize=12, label='Ogden')
plt.xlabel('Stretch')
plt.ylabel('Stress')
plt.title('Pure Shear Experiment')
plt.legend()
plt.show()